In [ ]:
# 必要なライブラリをインポートします
# PILは画像処理用、matplotlibは可視化用、pyocrはOCR用、numpyは数値演算用、cv2は画像操作用、globはファイルパス処理用です

from PIL import Image, ImageFilter, ImageOps
import matplotlib.pyplot as plt
import pyocr
import numpy as np
import cv2
import glob
import pandas as pd

# 画像をPIL形式からOpenCV形式に変換する関数
def cv2pil(image):
    new_image = image.copy()
    if new_image.ndim == 2:  # モノクロ画像の場合
        pass
    elif new_image.shape[2] == 3:  # カラー画像の場合
        new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
    elif new_image.shape[2] == 4:  # 透過色を含む画像の場合
        new_image = cv2.cvtColor(new_image, cv2.COLOR_BGRA2RGBA)
    new_image = Image.fromarray(new_image)
    return new_image

# OpenCV形式からPIL形式に変換する関数
def pil2cv(image):
    new_image = np.array(image, dtype=np.uint8)
    if new_image.ndim == 2:  # モノクロ画像の場合
        pass
    elif new_image.shape[2] == 3:  # カラー画像の場合
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif new_image.shape[2] == 4:  # 透過色を含む画像の場合
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    return new_image

# 文字列が数値に変換可能かどうかを判定する関数
def is_num(s):
    try:
        float(s)  # 文字列をfloat型に変換を試みる
    except ValueError:  # 変換できない場合はFalseを返す
        return False
    else:  # 変換できる場合はTrueを返す
        return True


In [ ]:
# 座標を特定するために，1つの画像を使用します．
img = cv2.imread('logger/frame_0002.jpg')
img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.gray()
im_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(im_gray)

In [ ]:
# 座標を特定します。数値は画像によって最適化する必要があります。
# [y:y+h, x:x+w]
img_tr1 = im_gray[20:140, 110:400]
plt.imshow(img_tr1)

In [ ]:
# 数値の読み取り精度を向上させるために、二値化します。
ret1, dst1 = cv2.threshold(img_tr1, 128, 255, cv2.THRESH_OTSU)
plt.imshow(dst1)
print(ret1)

In [ ]:
# 色の反転をします。数字部分を認識させるためです。
img_thresh1 = cv2.bitwise_not(img_tr1)
plt.imshow(img_thresh1)

In [ ]:
# 実際に数字を読み取ることができているかを確認するため、テストします。
tools = pyocr.get_available_tools()
tool = tools[0]

pil_image = Image.fromarray(img_thresh1)

txt = tool.image_to_string(
  pil_image,
  lang='jpn',
  builder=pyocr.builders.DigitBuilder(tesseract_layout=6)
)
print(txt)

In [ ]:
# 上記をまとめて処理をします。
# 画像ファイルのリストを取得し、ソートします。
files = glob.glob("image_file/*")
files.sort()

# 処理結果を保存するためのリストを初期化します。
cl_lst = []
file_lst = []

# OCRツールを取得します。
engines = pyocr.get_available_tools()
engine = engines[0]

# 各ファイルに対して処理を行います。
for file in files:
    # 画像を読み込み、グレースケールに変換します。
    img = cv2.imread(file)
    im_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 画像を特定の範囲でトリミングします。
    img_tr1 = im_gray[42:72, 450:500]
    img_tr2 = im_gray[73:105, 400:500]
    img_tr3 = im_gray[106:138, 400:500]
    img_tr4 = im_gray[172:200, 400:500]
    
    # 二値化を行い、読み取りやすくします。
    ret1, dst1 = cv2.threshold(img_tr1, 128, 255, cv2.THRESH_OTSU)
    ret2, dst2 = cv2.threshold(img_tr2, 128, 255, cv2.THRESH_OTSU)
    ret3, dst3 = cv2.threshold(img_tr3, 128, 255, cv2.THRESH_OTSU)
    ret4, dst4 = cv2.threshold(img_tr4, 128, 255, cv2.THRESH_OTSU)
    
    # 色の反転を行います。
    img_thresh1 = cv2.bitwise_not(dst1)
    img_thresh2 = cv2.bitwise_not(dst2)
    img_thresh3 = cv2.bitwise_not(dst3)
    img_thresh4 = cv2.bitwise_not(dst4)
    
    # 結果を保存します。
    a = file[1:]
    cv2.imwrite('result/' + a, img_thresh4)
    
    # 文字認識を行います。
    pil_image = Image.fromarray(img_thresh4)
    txt = engine.image_to_string(pil_image, lang="eng", builder=pyocr.builders.TextBuilder(tesseract_layout=6))
    
    # 結果を表示します。
    print("==========", file, "==========")
    print(txt)
    
    # 結果をリストに追加します。
    if not txt:
        cl_lst.append(-100)
    else:
        cl_lst.append(txt)
    file_lst.append(file)

cl = pd.DataFrame(cl_lst)
f_lst = pd.DataFrame(file_lst)
df = pd.concat([f_lst, cl], axis=1)
df.to_csv('data.csv')